In [1]:
# === RESUME EXTRACTION TỪ CHUNK 120 VỚI OPTIMIZED PROMPTS ===
!pip install -q langchain-groq chromadb pyvis pypdf sentence-transformers networkx pydantic python-dotenv

import os
import re
import time
import pickle
import unicodedata
import networkx as nx
from typing import List
from dotenv import load_dotenv

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import TokenTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
import chromadb
from chromadb.utils import embedding_functions
from pyvis.network import Network
from pydantic import BaseModel, Field

print("✅ Import thành công!")


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
c:\Users\pntha\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Import thành công!


In [2]:
# === LOAD CẤU HÌNH ===
load_dotenv()

GROQ_API_KEYS = [
    os.getenv("GROQ_API_KEY_1"),
    os.getenv("GROQ_API_KEY_2"),
    os.getenv("GROQ_API_KEY_3"),
    os.getenv("GROQ_API_KEY_4"),
    os.getenv("GROQ_API_KEY_5"),
    os.getenv("GROQ_API_KEY_6"),
    os.getenv("GROQ_API_KEY_7"),
    os.getenv("GROQ_API_KEY_8"),
    os.getenv("GROQ_API_KEY_9"),
]
PDF_PATH = os.getenv("PDF_PATH")

# Normalize functions
MEDICAL_ABBREVIATIONS = {
    "btm": "bệnh thận mạn", "tha": "tăng huyết áp", "đtđ": "đái tháo đường",
    "gfr": "độ lọc cầu thận", "egfr": "độ lọc cầu thận ước tính",
    "ckd": "bệnh thận mạn", "acei": "thuốc ức chế men chuyển",
}

MEDICAL_SYNONYMS = {
    "bệnh thận mạn": ["bệnh thận mãn", "suy thận mạn", "ckd"],
    "đái tháo đường": ["tiểu đường", "đtđ", "diabetes"],
    "tăng huyết áp": ["cao huyết áp", "tha"],
}

def normalize_medical_text(text: str) -> str:
    if not text: return "Unknown"
    text = unicodedata.normalize("NFC", text).strip().lower()
    text = re.sub(r'\s+', ' ', text)
    words = [MEDICAL_ABBREVIATIONS.get(re.sub(r'[^\w]', '', w), w) for w in text.split()]
    text = ' '.join(words)
    for canonical, variants in MEDICAL_SYNONYMS.items():
        for variant in variants:
            text = text.replace(variant, canonical)
    return re.sub(r'\s+', ' ', text).strip().title()

normalize_text = normalize_medical_text

class APIKeyManager:
    def __init__(self, api_keys: List[str]):
        self.api_keys = [k for k in api_keys if k and not k.startswith("gsk_YOUR")]
        self.current_index = 0
        self.failed_keys = set()
    def get_current_key(self) -> str:
        return self.api_keys[self.current_index]
    def rotate_key(self) -> bool:
        self.failed_keys.add(self.current_index)
        for i in range(len(self.api_keys)):
            next_index = (self.current_index + 1 + i) % len(self.api_keys)
            if next_index not in self.failed_keys:
                self.current_index = next_index
                print(f"🔄 Key #{next_index + 1}")
                return True
        return False
    def reset_failed(self):
        self.failed_keys.clear()

api_manager = APIKeyManager(GROQ_API_KEYS)
print(f"✅ {len(api_manager.api_keys)} API keys")

✅ 9 API keys


In [3]:
# === SCHEMA ===
class MedicalEntity(BaseModel):
    name: str
    type: str
    description: str = ""
    relevance_score: int = Field(default=5, ge=1, le=10)

class MedicalRelation(BaseModel):
    source_name: str
    target_name: str
    relation: str
    confidence_score: int = Field(default=5, ge=1, le=10)
    evidence: str = ""

class AMGExtractionResult(BaseModel):
    entities: List[MedicalEntity] = Field(default_factory=list)
    relations: List[MedicalRelation] = Field(default_factory=list)

print("✅ Schema ready")

✅ Schema ready


In [4]:
# === LOAD CHECKPOINT ===
print("📂 LOADING CHECKPOINT...\n")

# Load graph
graph_path = "./amg_data/graph_full.pkl"
if os.path.exists(graph_path):
    with open(graph_path, "rb") as f:
        G = pickle.load(f)
    print(f"✅ Graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
else:
    print("⚠️ Không tìm thấy checkpoint, tạo mới...")
    G = nx.DiGraph()



📂 LOADING CHECKPOINT...

✅ Graph: 479 nodes, 673 edges


In [ ]:
# === OPTIMIZED AMG EXTRACTOR ===
class OptimizedAMGExtractor:
    def __init__(self, api_manager: APIKeyManager):
        self.api_manager = api_manager
        
        # ★ OPTIMIZED PROMPT với COMPREHENSIVE Relation Types
        self.extraction_prompt = ChatPromptTemplate.from_messages([
            ("system", """Bạn là chuyên gia trích xuất thực thể y tế và quan hệ lâm sàng.

## ENTITY TYPES:
DISEASE, DRUG, SYMPTOM, TEST, ANATOMY, TREATMENT, PROCEDURE, RISK_FACTOR, LAB_VALUE, DOSAGE

## MEDICAL RELATION TYPES (20 LOẠI - TOÀN DIỆN):

### GROUP 1: DISEASE - SYMPTOM/OUTCOME
- HAS_SYMPTOM: bệnh có triệu chứng
- COMPLICATION_OF: biến chứng của bệnh
- ASSOCIATED_WITH: liên quan với bệnh

### GROUP 2: CAUSATION & RISK
- CAUSES: gây ra trực tiếp bệnh/triệu chứng
- INCREASES_RISK: tăng nguy cơ bệnh
- WORSENS: làm nặng tình trạng
- PREVENTS: phòng ngừa bệnh

### GROUP 3: DIAGNOSIS & MONITORING
- INDICATES: chỉ định bệnh/chẩn đoán
- REQUIRES_TEST: cần kiểm tra giám sát
- MONITORING_REQUIRED: cần theo dõi liên tục
- STAGES: giai đoạn của bệnh
- RISK_STRATIFICATION: phân tầng nguy cơ

### GROUP 4: TREATMENT
- TREATS: điều trị/cải thiện
- MANAGE_WITH: kết hợp điều trị
- SIDE_EFFECT_OF: tác dụng phụ của
- DOSAGE_ADJUSTMENT: điều chỉnh liều theo
- CONTRAINDICATED: chống chỉ định với

### GROUP 5: DRUG-DRUG INTERACTIONS
- CONTRAINDICATED_WITH: chống chỉ định kết hợp
- INTERACTION_WITH: tương tác
- METABOLIZED_BY: được chuyển hóa bởi

### GROUP 6: MISCELLANEOUS
- OTHER: quan hệ khác không phù hợp các loại trên (dùng khi có quan hệ nhưng không rõ loại)

## SCORING RUBRIC:
Entities - 9-10: CHÍNH | 7-8: QUAN TRỌNG | 5-6: PHỤ | 3-4: LIÊN QUAN | 1-2: RẤT PHỤ
Relations - 9-10: RÕ RÀNG | 7-8: CHẮC CHẮN | 5-6: NGHI NGỜ | <5: KHÔNG TRÍCH

## COMPREHENSIVE FEW-SHOT EXAMPLES:

### EXAMPLE 1: Disease with multiple relations
Input: "Bệnh thận mạn giai đoạn 5 gây phù chân, thiếu máu. HAS_SYMPTOM Xét nghiệm GFR < 15 chỉ định tình trạng này."
Output:
Entities:
- Bệnh thận mạn giai đoạn 5 (DISEASE, 10)
- Phù chân (SYMPTOM, 9)
- Thiếu máu (DISEASE, 8)
- GFR < 15 (LAB_VALUE, 10)

Relations:
- Bệnh thận mạn giai đoạn 5 → Xét nghiệm quạng [REQUIRES_TEST, 9]
- Bệnh thận mạn giai đoạn 5 → Thiếu máu [COMPLICATION_OF, 9]
### EXAMPLE 5: Other/Unknown relationships
Input: "Bệnh thận liên quan đến tình trạng này nhưng không rõ ràng."

### EXAMPLE 2: Drug treatment & side effects
- Bệnh thận (DISEASE, 9)
- Tình trạng (DISEASE, 6)

Relations:
- Bệnh thận → Tình trạng [OTHER, 5, "Liên quan nhưng không rõ loại"]
- Thiếu máu do suy thận (DISEASE, 9)
### EXAMPLE 2: Drug treatment & side effects
Input: "Erythropoietin 4000 unit/week điều trị thiếu máu do suy thận. Tác dụng phụ có thể là tăng huyết áp. Chống chỉ định nếu hemoglobin > 12 g/dL."
Output:
Entities:
- Erythropoietin (DRUG, 10)
- 4000 unit/week (DOSAGE, 8)
- Thiếu máu do suy thận (DISEASE, 9)
- Tăng huyết áp (SYMPTOM, 8)
- Hemoglobin > 12 (LAB_VALUE, 9)

Relations:
- Erythropoietin → Thiếu máu do suy thận [TREATS, 10]
- Erythropoietin → Tăng huyết áp [SIDE_EFFECT_OF, 8]
- Hemoglobin > 12 → Erythropoietin [CONTRAINDICATED, 9, "Nếu Hb > 12 không nên dùng"]
- Tăng kali máu (DISEASE, 9)
### EXAMPLE 3: Drug interactions
Input: "Cần tránh kết hợp ACE inhibitor và ARB vì tương tác tăng kali máu nguy hiểm. Lisinopril cần giảm 50% liều khi GFR < 30."
Output:
Entities:
- ACE inhibitor (DRUG, 9)
- ARB (DRUG, 9)
- Tăng kali máu (DISEASE, 9)
- Lisinopril (DRUG, 9)
- GFR < 30 (LAB_VALUE, 9)

Relations:
- ACE inhibitor → ARB [CONTRAINDICATED_WITH, 10, "Tương tác tăng kali"]
- Lisinopril → GFR < 30 [DOSAGE_ADJUSTMENT, 9, "Giảm 50% liều"]
- ACE inhibitor + ARB → Tăng kali máu [INTERACTION_WITH, 10]
- 3 tháng (DOSAGE, 7) [tần suất]
### EXAMPLE 4: Monitoring & prevention
Input: "Bệnh nhân CKD giai đoạn 4 cần theo dõi GFR 3 tháng/lần. Vaccin flu phòng ngừa pneumonia ở bệnh thận."
Output:
Entities:
- Bệnh thận mạn giai đoạn 4 (DISEASE, 9)
1. ✅ TRÍCH TẤT CẢ 21 LOẠI QUAN HỆ (20 + OTHER) nếu xuất hiện
2. ✅ PHÂN LOẠI CHÍNH XÁC: chọn loại quan hệ PHỤ HỢP NHẤT - chỉ dùng OTHER nếu thực sự không phù hợp
- Vaccin cúm (DRUG, 9)
4. ✅ Confidence dựa vào độ rõ ràng (9-10: rõ ràng, 5-6: gợi ý, OTHER thường 3-5)

6. ❌ KHÔNG trích: quan hệ mờ nhạt hoàn toàn (ngoại trừ OTHER với confidence < 5)
- Bệnh thận mạn giai đoạn 4 → GFR [MONITORING_REQUIRED, 9, "3 tháng/lần"]
- Vaccin cúm → Viêm phổi [PREVENTS, 8]

## CRITICAL RULES:
1. ✅ TRÍCH TẤT CẢ 20 LOẠI QUAN HỆ nếu xuất hiện
2. ✅ PHÂN LOẠI CHÍNH XÁC: chọn loại quan hệ PHỤ HỢP NHẤT
3. ✅ Evidence = dòng văn bản chứng minh quan hệ
4. ✅ Confidence dựa vào độ rõ ràng (9-10: rõ ràng, 5-6: gợi ý)
5. ❌ KHÔNG trích: số trang, quyết định, tên tác giả
6. ❌ KHÔNG trích: quan hệ mờ nhạt hoàn toàn
7. ✅ Ưu tiên: quan hệ y tế thực tế, có bằng chứng cụ thể

Trích xuất từ văn bản:"""),
            ("human", "{text}")
        ])
        
        self._init_llm()
    
    def _init_llm(self):
        self.llm = ChatGroq(temperature=0.1, model="llama-3.3-70b-versatile", 
                           api_key=self.api_manager.get_current_key())
        self.chain = self.extraction_prompt | self.llm.with_structured_output(AMGExtractionResult)
    
    def extract(self, text: str, max_retries=3):
        for attempt in range(max_retries):
            try:
                result = self.chain.invoke({"text": text})
                if result:
                    # Filter invalid entities
                    valid = [e for e in result.entities 
                            if len(e.name) >= 2 and 
                            not any(x in e.name.lower() for x in ['quyết định', 'văn bản', 'trang', 'bộ y tế'])]
                    result.entities = valid
                    
                    # Filter invalid relations - chỉ giữ quan hệ có confidence >= 5
                    valid_relations = [r for r in result.relations 
                                     if r.confidence_score >= 5 and 
                                     len(r.source_name) >= 2 and 
                                     len(r.target_name) >= 2]
                    result.relations = valid_relations
                
                return result if result else AMGExtractionResult()
            except Exception as e:
                if "rate" in str(e).lower() or "limit" in str(e).lower():
                    if self.api_manager.rotate_key():
                        self._init_llm()
print("✅ Optimized Extractor ready (21 Relation Types: 20 + OTHER)")

                    else:
        return AMGExtractionResult()print("✅ Optimized Extractor ready (20 Relation Types)")

                        print("⏳ Chờ 60s...")


                        time.sleep(60 * 5)
print("✅ Optimized Extractor ready (20 Relation Types)")        return AMGExtractionResult()

                        self.api_manager.reset_failed()                    time.sleep(1)

                        self._init_llm()                else:

✅ Optimized Extractor ready


In [6]:
# === HELPER FUNCTIONS ===
def add_entity_to_graph(G, entity: MedicalEntity, page_num: int, enhanced_desc="", conf_boost=0):
    """Add entity với deduplication và enhancement"""
    norm_name = normalize_medical_text(entity.name)
    base_conf = min(1.0, entity.relevance_score / 10.0)
    confidence = min(1.0, base_conf + conf_boost)
    
    full_desc = entity.description
    if enhanced_desc:
        full_desc += f" [Enhanced]: {enhanced_desc}"
    
    if not G.has_node(norm_name):
        G.add_node(norm_name, label=entity.name, type=entity.type.upper(),
                  description=full_desc, confidence=confidence,
                  relevance_score=entity.relevance_score, pages=[page_num])
    else:
        old_conf = G.nodes[norm_name].get('confidence', 0)
        if confidence > old_conf:
            G.nodes[norm_name]['confidence'] = confidence
            G.nodes[norm_name]['description'] = full_desc
        if page_num not in G.nodes[norm_name]['pages']:
            G.nodes[norm_name]['pages'].append(page_num)

def add_relation_to_graph(G, rel: MedicalRelation, page_num: int):
    src = normalize_medical_text(rel.source_name)
    tgt = normalize_medical_text(rel.target_name)
    if G.has_node(src) and G.has_node(tgt):
        G.add_edge(src, tgt, relation=rel.relation.upper(),
                  confidence=min(1.0, rel.confidence_score/10), 
                  evidence=rel.evidence, page=page_num)

def enhance_entity_from_vectordb(entity_name: str, col, n_results=2):
    """Query VectorDB để enhance entity"""
    try:
        results = col.query(query_texts=[entity_name], n_results=n_results)
        if results['documents'] and results['documents'][0]:
            combined = " ".join(results['documents'][0])[:300]
            return combined, 0.15
    except:
        pass
    return "", 0

print("✅ Helper functions ready")

✅ Helper functions ready


In [ ]:
# === RESUME EXTRACTION TỪ CHUNK 121 ===
START_CHUNK = 120  # Bạn dừng ở chunk 120
START_PAGE = 17

print(f"🚀 BẮT ĐẦU RESUME TỪ CHUNK {START_CHUNK + 1}...\n")

# Load PDF
loader = PyPDFLoader(PDF_PATH)
pages = loader.load()[START_PAGE-1:]
print(f"✂️ Cắt {START_PAGE-1} trang")

# Split chunks
splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(pages)
print(f"📦 Tổng chunks: {len(chunks)}")
print(f"▶️  Sẽ xử lý: chunk {START_CHUNK + 1} → {len(chunks)}\n")

# Extract
extractor = OptimizedAMGExtractor(api_manager)
chunks_to_process = chunks[START_CHUNK:]  # Bỏ qua 120 chunks đầu

print(f"🧠 ĐANG TRÍCH XUẤT (Optimized Prompts + Relations)...\n")

# Track stats
total_entities = 0
total_relations = 0

for i, chunk in enumerate(chunks_to_process, start=START_CHUNK+1):
    page_num = chunk.metadata.get("page", 0) + 1
    result = extractor.extract(chunk.page_content)
    
    if result and (len(result.entities) > 0 or len(result.relations) > 0):
        avg_rel = sum(e.relevance_score for e in result.entities) / len(result.entities) if result.entities else 0
        print(f"   Chunk {i}/{len(chunks)}: ", end="")
        
        # Add entities
        for ent in result.entities:
            add_entity_to_graph(G, ent, page_num, "", 0)
        total_entities += len(result.entities)
        print(f"+{len(result.entities)} ent ", end="")
        
        # Add relations
        rel_count_before = G.number_of_edges()
        for rel in result.relations:
            add_relation_to_graph(G, rel, page_num)
        rel_count_after = G.number_of_edges()
        new_relations = rel_count_after - rel_count_before
        total_relations += new_relations
        print(f"+{new_relations} rel", end="")
        
        if result.relations:
            print(f" [Avg conf: {sum(r.confidence_score for r in result.relations)/len(result.relations):.1f}]")
        else:
            print()
    else:
        print(f"   Chunk {i}/{len(chunks)}: --")
    
    # Save checkpoint mỗi 20 chunks
    if i % 20 == 0:
        with open("./amg_data/graph_full.pkl", "wb") as f:
            pickle.dump(G, f)
        print(f"   💾 Checkpoint: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
    
    time.sleep(1)  # Tránh rate limit

print(f"\n✅ HOÀN THÀNH!")
print(f"   Final: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
print(f"   📊 Stats: {total_entities} entities, {total_relations} relations extracted")

🚀 BẮT ĐẦU RESUME TỪ CHUNK 121...

✂️ Cắt 16 trang
📦 Tổng chunks: 408
▶️  Sẽ xử lý: chunk 121 → 408

🧠 ĐANG TRÍCH XUẤT (Optimized Prompts, NO Bidirectional)...

   Chunk 121/408: +7 entities (avg: 7.4)
   Chunk 122/408: +10 entities (avg: 8.3)
   Chunk 123/408: +12 entities (avg: 7.2)
   Chunk 124/408: +10 entities (avg: 6.7)
   Chunk 125/408: +6 entities (avg: 7.5)
🔄 Key #2
🔄 Key #3
🔄 Key #4
   Chunk 126/408: --
   Chunk 127/408: +13 entities (avg: 8.2)
   Chunk 128/408: +8 entities (avg: 7.8)
   Chunk 129/408: +10 entities (avg: 6.9)
   Chunk 130/408: +7 entities (avg: 7.7)
   Chunk 131/408: +8 entities (avg: 6.8)
   Chunk 132/408: +11 entities (avg: 7.4)
🔄 Key #5
   Chunk 133/408: +7 entities (avg: 7.4)
   Chunk 134/408: +5 entities (avg: 8.2)
   Chunk 135/408: +9 entities (avg: 7.9)
   Chunk 136/408: +11 entities (avg: 7.6)
   Chunk 137/408: +10 entities (avg: 7.6)
   Chunk 138/408: +5 entities (avg: 8.0)
🔄 Key #6
   Chunk 139/408: +16 entities (avg: 6.6)
   Chunk 140/408: +12 entit

In [ ]:
# === FREQUENCY BOOST (Entities + Relations) ===
print("\nÁP DỤNG FREQUENCY BOOST...\n")

# Boost Entities
boosted_entities = 0
for node in list(G.nodes):
    label = G.nodes[node].get('label', node)
    pages = G.nodes[node].get('pages', [])
    freq = len(pages)  # Số trang entity xuất hiện
    
    if freq >= 3:
        boost = 0.1 if freq >= 5 else 0.05
        old_conf = G.nodes[node]['confidence']
        G.nodes[node]['confidence'] = min(1.0, old_conf + boost)
        G.nodes[node]['description'] += f" [Freq: {freq}x]"
        boosted_entities += 1
        if boosted_entities <= 5:
            print(f"   Entity: {label}: {freq} pages → conf {old_conf:.2f} → {G.nodes[node]['confidence']:.2f}")

# Boost Relations (từ duplicate edges)
print(f"\n   Boosting Relations...")
edge_frequency = {}
for u, v, data in G.edges(data=True):
    key = (u, v, data.get('relation'))
    if key not in edge_frequency:
        edge_frequency[key] = {'count': 0, 'edges': []}
    edge_frequency[key]['count'] += 1
    edge_frequency[key]['edges'].append((u, v, data))

boosted_relations = 0
for (src, tgt, rel_type), info in edge_frequency.items():
    if info['count'] > 1:
        # Boost confidence for repeated relations
        for u, v, data in info['edges']:
            old_conf = data.get('confidence', 0.5)
            boost = min(0.1, info['count'] * 0.05)
            new_conf = min(1.0, old_conf + boost)
            G[u][v]['confidence'] = new_conf
            boosted_relations += 1

print(f"   ✅ Boosted {boosted_entities} entities, {boosted_relations} relations")
print(f"\n📊 Final Graph Stats:")
print(f"   - {G.number_of_nodes()} nodes")
print(f"   - {G.number_of_edges()} edges (relations)")


ÁP DỤNG FREQUENCY BOOST...

   Albumin: 7 pages → conf 0.90 → 1.00
   Creatinine: 7 pages → conf 0.90 → 1.00
   KDIGO: 7 pages → conf 0.80 → 0.90
   Bệnh thận mạn: 8 pages → conf 1.00 → 1.00
   Tắc nghẽn đường tiết niệu: 3 pages → conf 0.60 → 0.65

 Boosted 214 entities


In [11]:
# === SAVE FINAL GRAPH ===
with open("./amg_data/graph_full.pkl", "wb") as f:
    pickle.dump(G, f)

print(f"\n Đã lưu: ./amg_data/graph_full.pkl")
print(f"   - {G.number_of_nodes()} nodes")
print(f"   - {G.number_of_edges()} edges")


 Đã lưu: ./amg_data/graph_full.pkl
   - 1864 nodes
   - 743 edges


In [ ]:
# === VISUALIZE ===
print("\n CREATING VISUALIZATION...")

net = Network(height="850px", width="100%", directed=True, notebook=False)
net.force_atlas_2based(gravity=-50, spring_length=200)

color_map = {
    'DISEASE': '#ff6b6b', 'DRUG': '#4ecdc4', 'SYMPTOM': '#ffe66d',
    'TEST': '#95e1d3', 'ANATOMY': '#f38181', 'TREATMENT': '#aa96da',
    'RISK_FACTOR': '#ff6f61', 'LAB_VALUE': '#6bcae2', 'PROCEDURE': '#a8d8ea',
    'DOSAGE': '#fdbf6f'
}

# ★ 21 RELATION TYPES with distinctive colors
relation_color_map = {
    # Causation (RED)
    'CAUSES': '#e74c3c', 'INCREASES_RISK': '#c0392b', 'WORSENS': '#a93226',
    # Symptoms/Complications (ORANGE)
    'HAS_SYMPTOM': '#f39c12', 'COMPLICATION_OF': '#e67e22', 'ASSOCIATED_WITH': '#d68910',
    # Diagnosis (BLUE)
    'INDICATES': '#3498db', 'REQUIRES_TEST': '#2980b9', 'MONITORING_REQUIRED': '#1f618d',
    'STAGES': '#154360', 'RISK_STRATIFICATION': '#0b5394',
    # Treatment (GREEN)
    'TREATS': '#27ae60', 'MANAGE_WITH': '#229954', 'PREVENTS': '#1e8449',
    # Safety (PURPLE)
    'CONTRAINDICATED': '#9b59b6', 'SIDE_EFFECT_OF': '#8e44ad', 'DOSAGE_ADJUSTMENT': '#76448a',
    'CONTRAINDICATED_WITH': '#6c3483', 'INTERACTION_WITH': '#5b2c6f',
    # Drug Metabolism (PINK)
    'METABOLIZED_BY': '#e91e63',
    # Other (GRAY)
    'OTHER': '#95a5a6'
}

# Add nodes
for node, data in G.nodes(data=True):
    color = color_map.get(data.get('type', 'OTHER'), '#97C2FC')
    conf = data.get('confidence', 0.5)
    size = 15 + conf * 30
    label = data.get('label', node)
    title = f"<b>{label}</b><br>Type: {data.get('type')}<br>Conf: {conf:.2f}<br>Desc: {data.get('description', '')[:100]}"
    net.add_node(node, label=label, color=color, size=size, title=title)

# Add edges with relation labels
for u, v, data in G.edges(data=True):
    width = 1 + data.get('confidence', 0.5) * 4
    relation = data.get('relation', 'RELATED')
    color = relation_color_map.get(relation, '#666666')
    title = f"<b>{relation}</b><br>Conf: {data.get('confidence', 0.5):.2f}<br>Evidence: {data.get('evidence', '')[:80]}"
    net.add_edge(u, v, label=relation, width=width, color=color, 
                title=title, font={'size': 11, 'color': 'black'})

net.show_buttons(filter_=['physics'])
net.write_html("Medical_Graph_Full_Optimized.html")

print("\n ✅ Medical_Graph_Full_Optimized.html")
print(f"\n HOÀN TẤT! Đã trích xuất {G.number_of_nodes()} nodes và {G.number_of_edges()} relations.")
print("\n📊 RELATION TYPE LEGEND:")
print("   🔴 RED: Causation (CAUSES, INCREASES_RISK, WORSENS)")
print("   🟠 ORANGE: Symptoms/Complications (HAS_SYMPTOM, COMPLICATION_OF)")
print("   🔵 BLUE: Diagnosis (INDICATES, REQUIRES_TEST, MONITORING_REQUIRED)")
print("   🟢 GREEN: Treatment (TREATS, MANAGE_WITH, PREVENTS)")

print("   ⚫ GRAY: Other (OTHER - unclassified relationships)")
print("   🟣 PURPLE: Safety (CONTRAINDICATED, SIDE_EFFECT_OF, INTERACTION_WITH)")print("   🩷 PINK: Drug Metabolism (METABOLIZED_BY)")


 CREATING VISUALIZATION...
 Medical_Graph_Full_Optimized.html

 HOÀN TẤT!


## 📋 Hướng dẫn 20 loại quan hệ Y tế

### GROUP 1: DISEASE - SYMPTOM/OUTCOME
- **HAS_SYMPTOM**: Bệnh có triệu chứng → VD: "Bệnh thận → Phù chân"
- **COMPLICATION_OF**: Biến chứng của bệnh → VD: "Tăng kali máu ← Bệnh thận"
- **ASSOCIATED_WITH**: Liên quan không rõ nhân quả → VD: "Tiểu đường ↔ Bệnh thận"

### GROUP 2: CAUSATION & RISK
- **CAUSES**: Gây ra trực tiếp → VD: "Tiểu đường → Bệnh thận mạn"
- **INCREASES_RISK**: Tăng nguy cơ → VD: "Hút thuốc → ↑ nguy cơ ung thư"
- **WORSENS**: Làm nặng tình trạng → VD: "Tăng kali → Loạn nhịp tim"
- **PREVENTS**: Phòng ngừa → VD: "Vaccin cúm → Phòng Pneumonia"

### GROUP 3: DIAGNOSIS & MONITORING
- **INDICATES**: Chỉ định bệnh → VD: "GFR < 30 → Bệnh thận giai đoạn 5"
- **REQUIRES_TEST**: Cần kiểm tra → VD: "Bệnh thận → Xét nghiệm GFR"
- **MONITORING_REQUIRED**: Cần theo dõi liên tục → VD: "Bệnh thận giai đoạn 4 → GFR hàng 3 tháng"
- **STAGES**: Giai đoạn của bệnh → VD: "Giai đoạn 5 → Stage of: Bệnh thận mạn"
- **RISK_STRATIFICATION**: Phân tầng nguy cơ → VD: "GFR + Protein → High risk"

### GROUP 4: TREATMENT
- **TREATS**: Điều trị/cải thiện → VD: "Erythropoietin → Thiếu máu"
- **MANAGE_WITH**: Kết hợp điều trị → VD: "Thiếu máu → EPO + Sắt + Folic acid"
- **SIDE_EFFECT_OF**: Tác dụng phụ → VD: "Lisinopril → Ho khô (side effect)"
- **DOSAGE_ADJUSTMENT**: Điều chỉnh liều → VD: "GFR < 30 → Gentamicin giảm 50%"
- **CONTRAINDICATED**: Chống chỉ định → VD: "Hb > 12 → Chống chỉ định EPO"

### GROUP 5: DRUG-DRUG INTERACTIONS
- **CONTRAINDICATED_WITH**: Chống chỉ định kết hợp → VD: "ACE inhibitor + ARB → ✗"
- **INTERACTION_WITH**: Tương tác → VD: "Simvastatin + Erythromycin → ↑ rhabdomyolysis"
- **METABOLIZED_BY**: Được chuyển hóa → VD: "Warfarin → CYP2C9"


### GROUP 6: MISCELLANEOUS
- **OTHER**: Quan hệ khác không phù hợp các loại trên → VD: "Entity A liên quan Entity B nhưng không rõ loại"